In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [4]:
independent=dataset[["Age",	"EstimatedSalary",	"Gender_Male"]]
dependent=dataset["Purchased"]

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [6]:
#Preprocessing
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = [
    {'solver': ['liblinear'], 'penalty': ['l1', 'l2']},
    {'solver': ['newton-cg', 'lbfgs'], 'penalty': ['l2']},
    {'solver': ['saga'], 'penalty': ['l1', 'l2', 'elasticnet'], 'l1_ratio': [0.5]}
]

grid = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

grid.fit(X_train, Y_train)


Fitting 5 folds for each of 7 candidates, totalling 35 fits


GridSearchCV(estimator=LogisticRegression(max_iter=10000), n_jobs=-1,
             param_grid=[{'penalty': ['l1', 'l2'], 'solver': ['liblinear']},
                         {'penalty': ['l2'], 'solver': ['newton-cg', 'lbfgs']},
                         {'l1_ratio': [0.5],
                          'penalty': ['l1', 'l2', 'elasticnet'],
                          'solver': ['saga']}],
             scoring='f1_weighted', verbose=3)

In [8]:
#print best parameter
#TestSet can be predicted by final model
result=grid.cv_results_
grid_predictions=grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,grid_predictions)

from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,grid_predictions)

In [9]:
from sklearn.metrics import f1_score
f1_macro = f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'liblinear'}: 0.8794289739855382


In [10]:
print("Confusion Matrix:\n",cm)

Confusion Matrix:
 [[79  6]
 [10 39]]


In [11]:
print("The classification report:\n",clf_report)

The classification report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [12]:
#roc=r for receiver, o for operating, c for curve: auc= a for area, u for under, c for curve
from sklearn.metrics import roc_auc_score
#If probability is not given, then it won't work
roc_auc_score(Y_test, grid.predict_proba(X_test)[:,1])

0.9507803121248499

In [13]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004757,0.000611,0.007627,0.002068,l1,liblinear,NaN,"{'penalty': 'l1', 'solver': 'liblinear'}",0.804764,0.811321,0.66593,0.92351,0.922185,0.825542,0.094918,6
1,0.004330,0.001415,0.005800,0.001447,l2,liblinear,NaN,"{'penalty': 'l2', 'solver': 'liblinear'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
2,0.009510,0.001797,0.005265,0.000926,l2,newton-cg,NaN,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,3
3,0.007456,0.001866,0.005338,0.000692,l2,lbfgs,NaN,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,3
4,0.007572,0.001089,0.005408,0.001064,l1,saga,0.5,"{'l1_ratio': 0.5, 'penalty': 'l1', 'solver': '...",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
5,0.005811,0.001471,0.006816,0.001900,l2,saga,0.5,"{'l1_ratio': 0.5, 'penalty': 'l2', 'solver': '...",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,3
6,0.004535,0.000952,0.006171,0.000519,elasticnet,saga,0.5,"{'l1_ratio': 0.5, 'penalty': 'elasticnet', 'so...",0.782557,0.811321,0.68205,0.92351,0.922185,0.824325,0.091181,7
